In [ ]:
#default_exp evaluation.coco

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#COCO-evaluation" data-toc-modified-id="COCO-evaluation-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>COCO evaluation</a></span><ul class="toc-item"><li><span><a href="#COCO-conversion" data-toc-modified-id="COCO-conversion-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>COCO conversion</a></span></li><li><span><a href="#COCO-evaluator" data-toc-modified-id="COCO-evaluator-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>COCO evaluator</a></span></li></ul></li><li><span><a href="#Export" data-toc-modified-id="Export-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Export</a></span></li></ul></div>

# COCO evaluation
> Mostly copied from pytoch

In [ ]:
#export
from mantisshrimp.imports import *
from mantisshrimp.core import *
from mantisshrimp.evaluation.coco_eval import CocoEvaluator
from pycocotools.coco import COCO

## COCO conversion

In [ ]:
#export
def records2coco(records, id2cat):
    cats = [{'id':k, 'name':v} for k,v in id2cat.items()]
    annots = defaultdict(list)
    iinfos = []
    i = 0
    for r in tqdm(records):
        iinfos.append({
            'id': r.iinfo.iid,
            'file_name': r.iinfo.fp.name,
            'width': r.iinfo.w,
            'height': r.iinfo.h,
        })
        for annot in r.annot: 
            annots['id'].append(i) # TODO: Careful with ids! when over all dataset
            annots['image_id'].append(r.iinfo.iid)
            annots['category_id'].append(annot.oid)
            annots['bbox'].append(annot.bbox.pnts)
            annots['area'].append(annot.bbox.area)
            # TODO: for other types of masks
            if notnone(annot.seg): annots['segmentation'].extend(annot.seg.to_erle(r.iinfo.h, r.iinfo.w))
            annots['iscrowd'].append(annot.iscrowd)
            # TODO: Keypoints
            i += 1
    assert allequal(lmap(len, annots.values())), 'Mismatch lenght of elements'
    annots = [{k:v[i] for k,v in annots.items()} for i in range_of(annots['id'])]
    return {'images': iinfos, 'annotations': annots, 'categories': cats}

In [ ]:
#export
def coco_api_from_records(records, id2cat):
    coco_ds = COCO()
    coco_ds.dataset = records2coco(records, id2cat)
    coco_ds.createIndex()
    return coco_ds

## COCO evaluator

In [ ]:
#export
def _get_iou_types(model):
    model_without_ddp = model
    if isinstance(model, torch.nn.parallel.DistributedDataParallel):
        model_without_ddp = model.module
    iou_types = ["bbox"]
    if isinstance(model_without_ddp, torchvision.models.detection.MaskRCNN):
        iou_types.append("segm")
    if isinstance(model_without_ddp, torchvision.models.detection.KeypointRCNN):
        iou_types.append("keypoints")
    return iou_types

In [ ]:
#export
def COCOEvaluator(model, coco_ds):
    return CocoEvaluator(coco_ds, _get_iou_types(model))

# Export

In [ ]:
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted 02_data.core.ipynb.
Converted 04_data.annotations.ipynb.
Converted 06_transforms.ipynb.
Converted 07_data.load.ipynb.
Converted 08_models.ipynb.
Converted 11_evaluation.coco.ipynb.
Converted Untitled.ipynb.
Converted Untitled1.ipynb.
Converted index.ipynb.
